In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
path_to_file = '5_data.csv.zip'

In [11]:
retail = pd.read_csv(path_to_file, sep = ',', compression = 'zip', encoding='ISO-8859-1')
retail_columns = 

In [14]:
retail.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [17]:
retail.shape[0]-retail.drop_duplicates(subset=retail_columns).shape[0]

5268

In [ ]:
#Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. 
#Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 

#Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакци

In [24]:
retail = retail.drop_duplicates(subset=retail_columns)
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [25]:
retail.InvoiceNo.str.startswith('C').sum()

9251

In [ ]:
#Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [29]:
retail = retail[retail.Quantity > 0]
retail.shape

(526054, 8)

In [ ]:
#Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). 
#Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. 
#Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

#Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. 
#Для каждого заказа в данных может встречаться более 1 строки.

In [30]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [50]:
top_retail = retail[retail.Country == 'Germany'] \
    .groupby(['CustomerID'], as_index=False) \
    .agg({'InvoiceNo' : pd.Series.nunique})
germany_top = top_retail[top_retail.InvoiceNo > top_retail.InvoiceNo.quantile(0.8)].CustomerID

In [51]:
germany_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

In [ ]:
#Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения 
#и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

#Датафрейм записан в retail, нужные id пользователей – в germany_top

In [52]:
top_retail_germany = retail.query('CustomerID in @germany_top')

In [54]:
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


In [ ]:
#Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 

In [55]:
top_retail_germany.groupby(['StockCode'], as_index=False) \
    .agg({'InvoiceNo' : pd.Series.nunique}) \
    .sort_values('InvoiceNo', ascending = False)

,StockCode,InvoiceNo
1157,POST,213
409,22326,52
411,22328,38
453,22423,34
45,20719,30
...,...,...
520,22563,1
524,22569,1
528,22574,1
529,22576,1


In [ ]:
#Вернемся к анализу полного датасета retail. 
#Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

#Отфильтрованные данные записаны в retail.

In [57]:
retail = retail.assign(Revenue = retail.Quantity * retail.UnitPrice )

In [58]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


In [ ]:
#Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. 
#В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [65]:
retail.groupby(['InvoiceNo'], as_index=False) \
    .agg({'Revenue' : 'sum'}) \
    .sort_values('Revenue', ascending = False)[:5].InvoiceNo

20689    581483
2202     541431
17582    574941
18251    576365
9034     556444
Name: InvoiceNo, dtype: object

In [ ]:
transaction_data_updated.csv